In [1]:
import pandas as pd
import datetime

In [2]:
wines = pd.read_excel("data/Subset1.xls", 1)
print(min(wines['DOY']))
wines

213.0


,Table.No.,Cultivar,Color,Recommend,RecReg,Reg,Harvest date,DOY,Must.Brix,Must.Tacid,Must.pH,Wine.Alcohol,Wine.Facid,Wine.Extract,Wine.Tannin,Wine.Tasting
0,9,Chardony,White,Particularly recommended,1.0,3,1936-09-08,251.0,27.2,0.54,3.88,15.6,0.42,4.0,0.03,"Very distinct; good, but alcoholic"
1,9,Chardony,White,Particularly recommended,1.0,4,1936-08-31,243.0,23.5,0.67,3.79,13.0,0.39,NaN,NaN,Distinct; lacks acid; standard quality
2,9,Chardony,White,Particularly recommended,1.0,1,1937-10-01,273.0,22.1,0.72,3.38,12.0,0.52,2.0,0.03,Light body; fruity; distinct; good quality
3,9,Chardony,White,Particularly recommended,1.0,3,1937-09-11,253.0,24.6,0.63,3.69,12.8,0.49,2.1,0.03,Flat; tends to oxidize; but distinct
4,9,Chardony,White,Particularly recommended,1.0,1,1938-09-14,256.0,19.8,0.78,3.19,10.5,0.68,2.8,0.07,"Tart, but palatable; distinct; picked too early"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1478,25,Red Malaga (Molinera),Red,Table grape,NaN,4,1937-10-25,297.0,21.8,0.47,3.74,12.0,0.28,2.3,0.04,Very flat; some spoilage; very ordiry sherry
1479,25,Red Malaga (Molinera),Red,Table grape,NaN,4,1940-10-07,280.0,22.7,0.36,3.90,12.4,0.40,2.8,0.04,Flat; pink; very common; lacks everything
1480,25,Ribier,Red,Table grape,NaN,4,1935-11-01,304.0,20.2,0.55,NaN,10.8,0.34,2.5,0.08,"Flat; poor; sherry; soft, but no character"
1481,25,Ribier,Red,Table grape,NaN,4,1937-10-13,285.0,17.6,0.50,3.58,9.5,0.35,2.1,0.06,Flat; ordiry; tends to spoil


In [3]:
wines.dropna(subset = ['Harvest date'], inplace=True)
wines.reset_index(drop=True, inplace=True)

In [4]:
weather = pd.read_excel("data/Subset3.xls", [1,2,3,4,5])

In [5]:
weather[1]

,Year,Month,day,Tave,Tmax,Tmin,DOY,Unnamed: 7,Year.1,GST,WI,HI
0,1911,1,1,8.35,16.7,0.0,1,NaN,1911.0,15.073364,1099.050000,967.254052
1,1911,1,2,8.85,19.4,-1.7,2,NaN,1912.0,15.752336,1246.550000,1146.450272
2,1911,1,3,6.65,16.1,-2.8,3,NaN,1913.0,16.722430,1446.300000,1276.779404
3,1911,1,4,8.30,18.3,-1.7,4,NaN,1914.0,15.678579,1217.966001,1103.276910
4,1911,1,5,7.80,18.9,-3.3,5,NaN,1915.0,15.950196,1277.291993,1133.617078
...,...,...,...,...,...,...,...,...,...,...,...,...
36885,2011,12,27,6.95,15.6,-1.7,361,NaN,NaN,NaN,NaN,NaN
36886,2011,12,28,11.65,19.4,3.9,362,NaN,NaN,NaN,NaN,NaN
36887,2011,12,29,11.15,16.7,5.6,363,NaN,NaN,NaN,NaN,NaN
36888,2011,12,30,12.20,15.0,9.4,364,NaN,NaN,NaN,NaN,NaN


In [6]:
i = list(range(1,201))
i = list(map(str, i))
temperatures = pd.DataFrame(columns = i)

In [7]:
temperatures

,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200


In [8]:
for i in range(len(wines)):
    new_row = []
    r = wines["Reg"][i]
    d = wines['Harvest date'][i]
    h = int(wines['DOY'][i])
    new_row = list(weather[r].loc[(weather[r]['Year'] == d.year) & (weather[r]['DOY'].isin(range(h-200,h)))]['Tave'])
    if len(new_row) != 200:
        new_row = [float('nan')] * 200
    temperatures.loc[len(temperatures)] = new_row
print(temperatures.isnull().sum(axis = 0))

1      102
2      102
3      102
4      102
5      102
      ... 
196    102
197    102
198    102
199    102
200    102
Length: 200, dtype: int64


In [9]:
dataset = pd.concat([wines,temperatures], axis = 1)
dataset.describe()


,Table.No.,RecReg,Reg,DOY,Must.Brix,Must.Tacid,Must.pH,Wine.Alcohol,Wine.Facid,Wine.Extract,...,191,192,193,194,195,196,197,198,199,200
count,1473.000000,448.000000,1473.000000,1473.000000,1471.000000,1461.000000,1243.000000,1458.000000,1455.000000,1450.000000,...,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000
mean,17.843177,2.669643,3.390360,266.731161,22.638137,0.622327,3.498327,12.861728,0.488351,3.871241,...,20.567770,20.717736,20.624327,20.551278,20.468938,20.171097,20.067548,19.898622,19.852704,19.814041
std,3.840464,1.255954,1.247428,18.883646,2.304543,0.182924,0.279841,2.979097,0.130772,3.283334,...,3.769831,3.855531,3.874379,4.011755,4.071588,4.076647,4.026761,4.051500,3.974648,4.079995
min,9.000000,1.000000,1.000000,213.000000,14.200000,0.190000,2.760000,7.600000,0.200000,1.500000,...,8.050000,10.000000,8.350000,8.900000,7.750000,5.800000,6.950000,7.250000,7.750000,5.800000
25%,15.000000,2.000000,2.000000,255.000000,21.200000,0.500000,3.290000,10.900000,0.390000,2.300000,...,18.050000,18.237500,18.300000,17.750000,17.750000,17.500000,17.500000,17.087500,17.200000,17.012500
50%,19.000000,2.000000,4.000000,268.000000,22.700000,0.590000,3.490000,12.100000,0.480000,2.700000,...,20.000000,20.300000,20.300000,20.250000,19.875000,20.000000,19.734956,19.700000,19.700000,19.700000
75%,21.000000,4.000000,4.000000,280.000000,24.100000,0.720000,3.690000,13.400000,0.570000,3.100000,...,22.500000,22.578636,22.250000,22.750000,22.500000,22.200000,22.200000,21.930093,21.700000,21.950000
max,25.000000,5.000000,5.000000,336.000000,30.300000,1.630000,4.400000,22.200000,1.010000,18.500000,...,33.350000,33.350000,31.400000,31.950000,33.050000,33.350000,33.050000,33.350000,33.350000,31.400000


In [10]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1473 entries, 0 to 1472
Columns: 216 entries, Table.No. to 200
dtypes: datetime64[ns](1), float64(209), int64(2), object(4)
memory usage: 2.4+ MB


In [11]:
# Use pandas get_dummies() function to one-hot encode the 'fruits' column
one_hot_data = pd.get_dummies(dataset['Cultivar'])

# Concatenate the one-hot encoded DataFrame with the original DataFrame
dataset2 = pd.concat([dataset, one_hot_data], axis=1)

# Drop the original 'fruits' column
dataset2.drop('Cultivar', axis=1, inplace=True)

# Print the resulting DataFrame with one-hot encoded column
dataset2

,Table.No.,Color,Recommend,RecReg,Reg,Harvest date,DOY,Must.Brix,Must.Tacid,Must.pH,...,Trousseau,Valdepes,Verccia Sarda,Verccia bianca,Verdelho,Vermentino Favorita,Walschriesling,White Riesling,Zinfandel,sa Veltliner
0,9,White,Particularly recommended,1.0,3,1936-09-08,251.0,27.2,0.54,3.88,...,0,0,0,0,0,0,0,0,0,0
1,9,White,Particularly recommended,1.0,4,1936-08-31,243.0,23.5,0.67,3.79,...,0,0,0,0,0,0,0,0,0,0
2,9,White,Particularly recommended,1.0,1,1937-10-01,273.0,22.1,0.72,3.38,...,0,0,0,0,0,0,0,0,0,0
3,9,White,Particularly recommended,1.0,3,1937-09-11,253.0,24.6,0.63,3.69,...,0,0,0,0,0,0,0,0,0,0
4,9,White,Particularly recommended,1.0,1,1938-09-14,256.0,19.8,0.78,3.19,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,25,Red,Table grape,NaN,4,1937-10-25,297.0,21.8,0.47,3.74,...,0,0,0,0,0,0,0,0,0,0
1469,25,Red,Table grape,NaN,4,1940-10-07,280.0,22.7,0.36,3.90,...,0,0,0,0,0,0,0,0,0,0
1470,25,Red,Table grape,NaN,4,1935-11-01,304.0,20.2,0.55,NaN,...,0,0,0,0,0,0,0,0,0,0
1471,25,Red,Table grape,NaN,4,1937-10-13,285.0,17.6,0.50,3.58,...,0,0,0,0,0,0,0,0,0,0


problem : there are a lot of different categories so there are many dimensions added. We already had a dimension to high compared to the amount of observations so this pnly increases the problem.

In [12]:
# Use pandas get_dummies() function to one-hot encode the 'fruits' column
one_hot_data3 = pd.get_dummies(dataset['Color'])

# Concatenate the one-hot encoded DataFrame with the original DataFrame
dataset3 = pd.concat([dataset, one_hot_data3], axis=1)

# Drop the original 'fruits' column
dataset3.drop('Color', axis=1, inplace=True)

# Print the resulting DataFrame with one-hot encoded column
dataset3

,Table.No.,Cultivar,Recommend,RecReg,Reg,Harvest date,DOY,Must.Brix,Must.Tacid,Must.pH,...,193,194,195,196,197,198,199,200,Red,White
0,9,Chardony,Particularly recommended,1.0,3,1936-09-08,251.0,27.2,0.54,3.88,...,21.95,22.75,19.20,18.30,18.85,18.65,21.70,24.15,0,1
1,9,Chardony,Particularly recommended,1.0,4,1936-08-31,243.0,23.5,0.67,3.79,...,19.75,20.55,22.25,23.60,22.75,22.25,20.80,23.05,0,1
2,9,Chardony,Particularly recommended,1.0,1,1937-10-01,273.0,22.1,0.72,3.38,...,18.60,16.40,19.15,21.70,21.65,18.30,16.95,16.35,0,1
3,9,Chardony,Particularly recommended,1.0,3,1937-09-11,253.0,24.6,0.63,3.69,...,18.60,17.20,19.45,20.85,20.80,17.25,17.75,21.65,0,1
4,9,Chardony,Particularly recommended,1.0,1,1938-09-14,256.0,19.8,0.78,3.19,...,20.00,20.85,20.30,20.55,21.65,22.50,21.95,20.85,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,25,Red Malaga (Molinera),Table grape,NaN,4,1937-10-25,297.0,21.8,0.47,3.74,...,18.35,19.40,21.10,18.30,21.10,20.25,20.25,17.75,1,0
1469,25,Red Malaga (Molinera),Table grape,NaN,4,1940-10-07,280.0,22.7,0.36,3.90,...,18.35,23.05,21.15,18.90,14.45,13.60,16.40,16.95,1,0
1470,25,Ribier,Table grape,NaN,4,1935-11-01,304.0,20.2,0.55,NaN,...,15.80,12.20,13.05,15.00,15.00,11.95,7.75,5.85,1,0
1471,25,Ribier,Table grape,NaN,4,1937-10-13,285.0,17.6,0.50,3.58,...,13.85,16.40,18.05,20.55,19.75,19.20,19.75,18.90,1,0


In [13]:
mapping = {'White': -1, 'Red': 1}

# Apply the mapping to the 'category' column using the map() function
dataset['Color'] = dataset['Color'].map(mapping)

dataset

,Table.No.,Cultivar,Color,Recommend,RecReg,Reg,Harvest date,DOY,Must.Brix,Must.Tacid,...,191,192,193,194,195,196,197,198,199,200
0,9,Chardony,-1,Particularly recommended,1.0,3,1936-09-08,251.0,27.2,0.54,...,18.90,20.85,21.95,22.75,19.20,18.30,18.85,18.65,21.70,24.15
1,9,Chardony,-1,Particularly recommended,1.0,4,1936-08-31,243.0,23.5,0.67,...,17.50,16.65,19.75,20.55,22.25,23.60,22.75,22.25,20.80,23.05
2,9,Chardony,-1,Particularly recommended,1.0,1,1937-10-01,273.0,22.1,0.72,...,20.55,18.90,18.60,16.40,19.15,21.70,21.65,18.30,16.95,16.35
3,9,Chardony,-1,Particularly recommended,1.0,3,1937-09-11,253.0,24.6,0.63,...,18.30,17.50,18.60,17.20,19.45,20.85,20.80,17.25,17.75,21.65
4,9,Chardony,-1,Particularly recommended,1.0,1,1938-09-14,256.0,19.8,0.78,...,18.60,19.70,20.00,20.85,20.30,20.55,21.65,22.50,21.95,20.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,25,Red Malaga (Molinera),1,Table grape,NaN,4,1937-10-25,297.0,21.8,0.47,...,16.10,15.30,18.35,19.40,21.10,18.30,21.10,20.25,20.25,17.75
1469,25,Red Malaga (Molinera),1,Table grape,NaN,4,1940-10-07,280.0,22.7,0.36,...,21.65,16.70,18.35,23.05,21.15,18.90,14.45,13.60,16.40,16.95
1470,25,Ribier,1,Table grape,NaN,4,1935-11-01,304.0,20.2,0.55,...,15.00,14.45,15.80,12.20,13.05,15.00,15.00,11.95,7.75,5.85
1471,25,Ribier,1,Table grape,NaN,4,1937-10-13,285.0,17.6,0.50,...,16.15,13.60,13.85,16.40,18.05,20.55,19.75,19.20,19.75,18.90


In [14]:
dataset.nunique(axis=0)

Table.No.     17
Cultivar     148
Color          2
Recommend      5
RecReg         5
            ... 
196          189
197          185
198          185
199          185
200          190
Length: 216, dtype: int64

In [15]:
from sklearn.feature_extraction import FeatureHasher


# Instantiate a FeatureHasher object with the desired number of output features (n_features)
hasher = FeatureHasher(n_features=148, input_type='string')

# Perform feature hashing on the 'fruits' column
hashed_features = hasher.fit_transform(dataset['Cultivar'])

# Convert the hashed features to a dense NumPy array and create a DataFrame
hashed_features_df = pd.DataFrame(hashed_features.toarray())

# Concatenate the hashed features DataFrame with the original DataFrame
dataset4 = pd.concat([dataset, hashed_features_df], axis=1)

# Drop the original 'fruits' column
dataset4.drop('Cultivar', axis=1, inplace=True)

dataset4


ModuleNotFoundError: No module named 'sklearn'